In [1]:
import numpy as np
import pandas as pd 
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.shape

(569, 33)

In [6]:
df.drop(columns=['id', 'Unnamed: 32'],inplace=True)

In [7]:
df.shape

(569, 31)

### train_test_split

In [8]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=.2)

### scaling

In [11]:
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Label Encoding

In [12]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

###  Numpy arrays to PyTorch tensors

In [13]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [14]:
X_train_tensor.shape

torch.Size([455, 30])

In [15]:
y_train_tensor.shape

torch.Size([455])

In [31]:
from torch.utils.data import Dataset,DataLoader

class CustomDataset(Dataset):

    def __init__(self,features,labels):
        self.features=features
        self.labels=labels


    def __len__(self):
        return len(self.features)
    

    def __getitem__(self, idx):
        return self.features[idx],self.labels[idx]

In [32]:
train_dataset=CustomDataset(X_train_tensor,y_train_tensor)
test_dataset=CustomDataset(X_test_tensor,y_test_tensor)

In [33]:
train_dataset[10]

(tensor([-0.3570,  0.0187, -0.3386, -0.4261, -0.0647,  0.1318, -0.2451, -0.5278,
         -0.1634,  0.3548, -0.5060, -1.1561, -0.2241, -0.4368, -0.4504,  0.3716,
          0.5073, -0.3562,  0.8791,  0.2982, -0.4127, -0.5855, -0.2710, -0.4549,
         -0.1773,  0.4180,  0.3255, -0.2710,  1.1382,  0.3966]),
 tensor(0.))

In [34]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [39]:
import torch.nn as nn

class MySimpleNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()
        self.linear=nn.Linear(num_features,1)
        self.sigmoid=nn.Sigmoid()


    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)

        return out



### Important Parameters

In [40]:
learning_rate = 0.1
epochs = 25

In [41]:
# create model
model=MySimpleNN(X_train_tensor.shape[1])

# define optimiser
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

# define loss fuction
loss_function=nn.BCELoss()

### Training pipeline

In [42]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.35545632243156433
Epoch: 2, Loss: 0.05927363038063049
Epoch: 3, Loss: 0.061623357236385345
Epoch: 4, Loss: 0.05495326966047287
Epoch: 5, Loss: 0.17793121933937073
Epoch: 6, Loss: 0.21018922328948975
Epoch: 7, Loss: 0.028993096202611923
Epoch: 8, Loss: 0.18037374317646027
Epoch: 9, Loss: 0.0443439818918705
Epoch: 10, Loss: 0.26816004514694214
Epoch: 11, Loss: 0.010510089807212353
Epoch: 12, Loss: 0.23397491872310638
Epoch: 13, Loss: 0.018948284909129143
Epoch: 14, Loss: 0.0782698318362236
Epoch: 15, Loss: 0.018211202695965767
Epoch: 16, Loss: 0.1392907351255417
Epoch: 17, Loss: 0.061148036271333694
Epoch: 18, Loss: 0.0839533880352974
Epoch: 19, Loss: 0.031815752387046814
Epoch: 20, Loss: 0.1304043084383011
Epoch: 21, Loss: 0.07649052143096924
Epoch: 22, Loss: 0.07668181508779526
Epoch: 23, Loss: 0.1240304708480835
Epoch: 24, Loss: 0.001377411768771708
Epoch: 25, Loss: 0.061256613582372665


### Evaluation

In [43]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')



Accuracy: 0.9271
